# Example usage of the Hybrid Model

In [1]:
# imports
import jax
import jax.numpy as jnp

import jax_cfd.base.equations as eqtns
import jax_cfd.base as cfd
import jax_cfd.base.grids as grids

import jax_cfd.base.advection as advection
import jax_cfd.base.diffusion as diffusion
from jax_cfd.base import finite_differences as fd

import numpy as np #?

import xarray
import seaborn
import pickle

import haiku as hk


import jax_cfd.ml.newSaveObject as saving
import jax_cfd.ml.diego_towers as mytowers

In [12]:
# this allows me to reload a module without having to interrupt the kernel
import importlib
importlib.reload(eqtns)

<module 'jax_cfd.base.equations' from '/rds/general/user/dd519/home/FYP/forked_jax/jax-cfd/jax_cfd/base/equations.py'>

In [67]:
## forcings

## Here are different types of forcing functions/conditions

# Linear force due to uniform pressure gradient
def pressure_gradient_forcing(pressure_gradient: float):
  
  def forcing(v):
    force_vector = (pressure_gradient, 0)
    return tuple(cfd.grids.GridArray(f * jnp.ones_like(u.data), u.offset, u.grid)
                 for f, u in zip(force_vector, v))
  
  return forcing

# Turbulent forcing (kolmogorov forcing)
def turbulentForcing(grid):
    return cfd.forcings.simple_turbulence_forcing(grid, constant_magnitude = 0.5, linear_coefficient = -0.8) # params specified in jax-cfd for 2D turbulence

In [68]:
# intialising problem

#grid
#bcs
#initial velocities
#forcing

size = (256,64)
domain = ((0, 8), (0, 2))

density = 1. ### original 1.
viscosity = 1e-4 # kinematic viscosity ### original 1e-3

pressure_gradient = 2e-3  # uniform dP/dx ### original 2e-3


grid = cfd.grids.Grid(size, domain = domain)


velocity_bc = (cfd.boundaries.channel_flow_boundary_conditions(ndim=2),
               cfd.boundaries.channel_flow_boundary_conditions(ndim=2))

pressure_solve = cfd.pressure.solve_fast_diag_channel_flow ### solve_fast_diag_channel_flow OR solve_cg

vx_fn = lambda x, y: jnp.zeros_like(x) 
vy_fn = lambda x, y: jnp.zeros_like(x)

max_velocity = 1.
cfl_safety_factor = 0.5

# time steps per output
# inner_steps = 1  ## defined below

# number of outputs
# outer_steps = 1000   ## defined below to allow the toTime method to work

forcing = lambda pressure_gradient, grid :  cfd.forcings.sum_forcings(
        pressure_gradient_forcing(pressure_gradient),
        #turbulentForcing(grid),
        cfd.forcings.taylor_green_forcing(grid, scale = 0.05)
        #cfd.forcings.kolmogorov_forcing(grid)
        #cfd.forcings.linear_forcing(grid,1.01)
        #cfd.forcings.filtered_forcing(grid = grid, spectral_density = 1)
        #cfd.forcings.filtered_linear_forcing(grid=grid, upper_wavenumber = 2, coefficient = 1, lower_wavenumber = 1)
)

#######################
# # Evaluate forcing and grid
# for i in range(len(simSpecs)):
#     simSpecs[i]['grid'] = grid = cfd.grids.Grid(simSpecs[i]["size"], 
#                                                               domain=simSpecs[i]["domain"])
#     simSpecs[i]['forcing'] = simSpecs[i]['forcing'](pressure_gradient = simSpecs[i]['pressure_gradient'], 
#                                          grid = simSpecs[i]['grid']
#       )
#######################


forcing = forcing(pressure_gradient = pressure_gradient, 
                  grid = grid)

In [79]:
# finish setting up simulation
v0 = cfd.initial_conditions.initial_velocity_field(
       velocity_fns = (vx_fn, vy_fn),
       grid = grid,
       velocity_bc = velocity_bc,
       pressure_solve = pressure_solve,
       iterations=5)




## divergence check
div = cfd.finite_differences.divergence(v0)
np.testing.assert_allclose(div.data, 0)

# calculate stable timestep (do we need to save this?)
dt = cfd.equations.stable_time_step(max_velocity, 
                                    cfl_safety_factor, 
                                    viscosity, 
                                    grid)

dt

0.015625

In [138]:
# time steps per output
inner_steps = 200

# Option ONE
outer_steps = 10

# Option TWO
toTime = False
if toTime:
    target_sim_time = 15 #in time
    outer_steps = target_sim_time//(inner_steps*dt)
    
outer_steps

10

In [139]:
print("Total simulation time: {:.2f}s".format(inner_steps*outer_steps*dt))

Total simulation time: 31.25s


In [140]:
## define convection function
def convect(v):
  return tuple(
      cfd.advection.advect_van_leer(u, v, dt) for u in v)

## define diffusion function

def diffuse(c, nu):
  """Returns the rate of change in a concentration `c` due to diffusion."""
  return nu * fd.laplacian(c)


In [141]:
def ConvNet(x):
    cnn = mytowers.CNN(CNN_specs)
    return cnn(x)

In [142]:
##import models to correct diffusion and convection
# Load corrector/inference network


save_path = "./models/correctors/laplacian_correctors/third_laplacian.pickle"
with open(save_path,'rb',) as pickle_file:
    mylaplacian = pickle.load(pickle_file)
    CNN_specs = mylaplacian.CNN_specs
    mylaplacian.forward_pass = hk.without_apply_rng(hk.transform(ConvNet))
    
def diffuse(c,nu):
    return nu * mylaplacian.forward_pass.apply(mylaplacian.params,c)

## Defining and choosing from a number of different numerical step functions

In [143]:
# define function to advance one timestep

# Semi-implicit Navier Stokes
NS_step_functions = {    
    "navier_stokes_explicit_terms" : cfd.equations.navier_stokes_explicit_terms(
                                                density=density,
                                                viscosity=viscosity,
                                                dt=dt,
                                                grid=grid,
                                                convect=convect,
                                                forcing= forcing
    ),
    
    "semi_implicit_navier_stokes" : cfd.equations.semi_implicit_navier_stokes(
                                                density=density,
                                                viscosity=viscosity,
                                                dt=dt,
                                                grid=grid,
                                                convect=convect,
                                                pressure_solve = pressure_solve,
#                                                 diffuse = diffuse,
                                                forcing= forcing
    ),
    "corrected_semi_implicit_navier_stokes" : cfd.equations.corrected_semi_implicit_navier_stokes(
                                                density=density,
                                                viscosity=viscosity,
                                                dt=dt,
                                                grid=grid,
                                                convect=convect,
                                                pressure_solve = pressure_solve,
                                                diffuse = diffuse,
                                                forcing= forcing
    ),
    
    "implicit_diffusion_navier_stokes" : cfd.equations.implicit_diffusion_navier_stokes(
                                                density=density,
                                                viscosity=viscosity,
                                                dt=dt,
                                                grid=grid,
                                                convect=convect,
                                                pressure_solve = pressure_solve,
                                                forcing= forcing
    )
        
}


# NS_step = NS_step_functions["semi_implicit_navier_stokes"]
NS_step = NS_step_functions["corrected_semi_implicit_navier_stokes"]

## Defining the ML section of the solver


In [144]:
jax.config.update("jax_disable_jit", False)
single_step = cfd.funcutils.repeated(NS_step,steps=inner_steps)

In [145]:
# jax.jit function enables execution on CPU/GPU/TPU
rollout_fn = cfd.funcutils.trajectory(
    single_step, outer_steps, start_with_input=True)

In [ ]:
## compute trajectory
%time _, trajectory = jax.device_get(rollout_fn(v0))

In [ ]:
# load into xarray for visualization and analysis
ds = xarray.Dataset(
    {
        'u': (('time', 'x', 'y'), trajectory[0].data),
        'v': (('time', 'x', 'y'), trajectory[1].data),
    },
    coords={
        'x': grid.axes()[0],
        'y': grid.axes()[1],
        'time': dt * inner_steps * np.arange(1, outer_steps + 1)
    }
)

In [ ]:
def vx_at_x_upper_boundary(ds):
  return ds.u.isel(x=-1)  # slice operator

p = ds.pipe(vx_at_x_upper_boundary).plot(hue='time')

In [ ]:
seaborn.heatmap(ds.u.isel(time=-1).T)